In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [3]:
print(train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [4]:
train.target[0]

7

In [5]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

In [8]:
x_train = tfidf.fit_transform(train.data)

In [9]:
x_train.shape

(11314, 10000)

In [11]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(train.target)

In [12]:
train.target[:5]

array([ 7,  4,  4,  1, 14])

In [13]:
y_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [14]:
x_test = tfidf.transform(test.data)
y_test = lb.transform(test.target)

In [15]:
import joblib

joblib.dump(
    {
        'tfidf': tfidf,
        'target_names': train.target_names,
        'x_train': x_train,
        'y_train': y_train,    
        'x_test': x_test,
        'y_test': y_test,
    },
    'newsgroup.pkl')

['newsgroup.pkl']

In [16]:
import joblib
data = joblib.load('newsgroup.pkl')
locals().update(data)

In [17]:
import tensorflow as tf

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='softmax')
])

In [19]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01),
    metrics=['accuracy'])

In [20]:
model.fit(
    x_train.toarray(),
    y_train,
    epochs=30,
    validation_split=.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')],
)

Epoch 1/30
319/319 [==============================] - 2s 5ms/step - loss: 1.6778 - accuracy: 0.7883 - val_loss: 0.9219 - val_accuracy: 0.8860
Epoch 2/30
319/319 [==============================] - 1s 4ms/step - loss: 0.5220 - accuracy: 0.9616 - val_loss: 0.5953 - val_accuracy: 0.9002
Epoch 3/30
319/319 [==============================] - 1s 4ms/step - loss: 0.2756 - accuracy: 0.9829 - val_loss: 0.4770 - val_accuracy: 0.9046
Epoch 4/30
319/319 [==============================] - 1s 4ms/step - loss: 0.1716 - accuracy: 0.9906 - val_loss: 0.4198 - val_accuracy: 0.9090
Epoch 5/30
319/319 [==============================] - 1s 4ms/step - loss: 0.1161 - accuracy: 0.9947 - val_loss: 0.3840 - val_accuracy: 0.9064


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                200020    
Total params: 200,020
Trainable params: 200,020
Non-trainable params: 0
_________________________________________________________________


In [22]:
import pandas as pd

In [23]:
w, _ = model.weights

In [24]:
weights = pd.DataFrame(w.numpy(), columns=target_names, index=tfidf.get_feature_names())

In [26]:
weights.head()

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
00,-2.674435,-1.035579,-0.994162,-0.465925,-1.729387,-0.015489,1.571029,-0.646751,-0.704679,0.582772,-0.286080,-1.762668,-0.851056,-1.445516,-0.955691,-0.321742,-0.216436,-0.580323,-1.067068,-2.021615
000,0.044126,-0.987592,-0.679350,-3.813828,-2.542926,-1.642839,0.568073,0.199159,-0.961499,0.560883,-0.980337,-1.238528,-0.171723,-1.524678,0.455859,-2.267924,0.610281,0.487733,0.261342,-0.752898
005,-0.401186,-0.576127,-0.419901,-0.498370,-0.462460,-0.320004,-0.461909,-0.419471,-0.583020,-0.673069,0.366597,-0.430570,0.249441,-0.551177,-0.664298,-0.613359,-0.335992,-0.475428,-0.545661,-0.400229
01,0.115549,0.322201,-3.210002,-0.149213,-0.194841,-1.229748,-1.066886,-1.022436,-0.849613,0.184199,-2.157596,0.580046,-1.003356,-0.787828,-0.996781,1.115343,-0.408665,-0.625900,-1.787646,-0.298261
02,-0.144768,-1.139957,-0.146742,0.146182,-1.065380,0.110873,-0.408038,-0.596715,-1.274021,-0.008528,-1.608270,-0.419521,0.272188,-0.985650,-0.185783,0.568964,-1.789925,-1.446940,0.050327,0.598387


In [27]:
column = 'sci.space'
weights.sort_values(column)[[column]].tail(10)

,sci.space
launch,1.685444
nasa,1.750745
pat,1.818211
shuttle,1.851114
prb,1.978014
moon,2.096910
earth,2.188098
sci,2.321253
orbit,2.616498
space,3.383337


Dropout 레이어를 넣은 다층신경망 모형이다.

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, 'relu'),    
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation='softmax')
])

Batch Normalization 레이어를 넣은 다층신경망 모형이다.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='softmax')
])

만약 활성화 함수를 적용하기 전에 batch normalization을 적용하려면 다음과 같이 한다.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(20, activation='softmax')
])

텐서플로에서는 원핫인코딩을 하지 않아도 다항분류를 할 수 있다. 먼저 원핫인코딩 되기 이전의 형태로 되돌린다.

In [29]:
import numpy as np

In [40]:
#벡터 (0,1,2,⋯,18,19)를 곱하면 각 행에서 1이 있는 위치의 값만 남는다.
train_target = y_train @ np.arange(20)

In [38]:
#또는 각 행에서 가장 큰 값의 위치를 찾아도 된다.
train_target = np.argmax(y_train, axis=1)

In [41]:
#원핫인코딩 이전의 형태로 되돌렸다.
train_target[:5]

array([ 7,  4,  4,  1, 14])

손실 함수를 sparse_categorical_crossentropy로 바꾼다.

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.1),
    metrics=['accuracy'])

y_train 대신 train_target으로 학습을 진행한다.

In [ ]:
model.fit(
    x_train.toarray(),
    train_target,
    epochs=30,
    validation_split=.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')],
)

레이블 스무딩을 하는 방법에는 두 가지가 있다. 하나는 데이터를 스무딩된 형태로 바꾸는 것이다.

In [ ]:
e = 0.2

In [ ]:
y_smooth = y_train * (1 - e) + (e / 20)
y_smooth

학습을 시킬 때 y_train 대신 y_smooth를 사용하면 된다.

또는 손실 함수 CategoricalCrossentropy에서 label_smoothing을 설정해주어도 된다.

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2),
    optimizer='adam',
    metrics=['accuracy'])